In [35]:
# === Solar Atlas: 후보지 + 기존 설비 + 깔끔 대시보드 (ONE FILE) ===
import pandas as pd
import folium
from folium import DivIcon
from folium.plugins import MarkerCluster, HeatMap, Fullscreen
import branca.colormap as cm
import webbrowser
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

# ----------------------------------------------------
# 0) 데이터 적재 & 전처리 유틸
#    - 컬럼명에서 불필요한 공백/제어문자를 제거(NFKC 정규화 포함)
#    - 실제 설비 목록(충남/경북/경남) 엑셀을 읽어들임
# ----------------------------------------------------
def normalize_columns(df):
    """DataFrame 컬럼명을 통일된 형태로 정리한다."""
    df = df.copy()
    df.columns = (
        df.columns.astype(str)
        .str.normalize("NFKC")                 # 유니코드 정규화(전각/반각 등)
        .str.replace("\u00A0", " ", regex=False)  # NBSP 제거
        .str.replace("\u3000", " ", regex=False)  # 전각 공백 제거
        .str.replace(r"[\r\n\t]", "", regex=True) # 개행/탭 등 제어문자 제거
        .str.strip()                              # 양끝 공백 제거
    )
    return df

# 각 시도 설비 목록 파일(경로만 맞추면 바로 동작)
df_cn = normalize_columns(pd.read_excel("dataset/충남_설비목록.xlsx"))
df_gb = normalize_columns(pd.read_excel("dataset/경북_설비목록.xlsx"))
df_gn = normalize_columns(pd.read_excel("dataset/경남_설비목록.xlsx"))

# ----------------------------------------------------
# 1) 후보지(수기 선정)
#    - 좌표(lat, lon), 간단 메모, 대표 일사량(MJ/m², 예시값) 포함
# ----------------------------------------------------
sites = [
    {"광역":"충남","지역":"논산시","lat":36.1839054,"lon":127.1446669,"메모":"농업지역, 넓은 평지","일사량":4700},
    {"광역":"충남","지역":"예산군","lat":36.6917263,"lon":126.8141959,"메모":"삽교천 유역 평야","일사량":4650},
    {"광역":"경북","지역":"포항시 남구","lat":36.0065257,"lon":129.3890059,"메모":"일사량 양호, 산업단지 활용","일사량":5000},
    {"광역":"경북","지역":"구미시","lat":36.2117565,"lon":128.3457108,"메모":"낙동강 평지, 수요지 인접","일사량":4900},
    {"광역":"경남","지역":"창녕군","lat":35.4654473,"lon":128.4737870,"메모":"온화한 기후, 평지","일사량":4850},
    {"광역":"경남","지역":"김해시","lat":35.2495439,"lon":128.8422028,"메모":"일사량 양호, 대도시 인접","일사량":4950},
]
# 광역 컬러(후보지 기본 색상 대체용)
region_color = {"경북":"#d62728","경남":"#1f77b4","충남":"#2ca02c"}

# ----------------------------------------------------
# 2) 좌표 보정 & 컬럼 선택 유틸
#    - 위/경도 열이 뒤바뀐 행을 현장에서 자주 만나므로, 각 행 단위로 안정적으로 보정
#    - 다양한 열 후보 중 실제 df에 있는 첫 번째 컬럼명을 선택하는 pick_col
# ----------------------------------------------------
def _is_lat(v):
    """대한민국 대략 위도 범위(33~39도) 판단"""
    return pd.notna(v) and 33 <= float(v) <= 39

def _is_lon(v):
    """대한민국 대략 경도 범위(124~132도) 판단"""
    return pd.notna(v) and 124 <= float(v) <= 132

def normalize_latlon(df, lat_col="위도", lon_col="경도"):
    """
    df의 위도/경도 열이 뒤바뀐 행을 자동 보정.
    유효 범위가 아니면 NaN으로 처리.
    """
    lat_raw = pd.to_numeric(df[lat_col], errors="coerce")
    lon_raw = pd.to_numeric(df[lon_col], errors="coerce")
    lat_fix, lon_fix = [], []
    for la, lo in zip(lat_raw, lon_raw):
        if _is_lat(la) and _is_lon(lo):            # 정상 케이스
            lat_fix.append(float(la)); lon_fix.append(float(lo))
        elif _is_lat(lo) and _is_lon(la):          # 뒤바뀐 케이스
            lat_fix.append(float(lo)); lon_fix.append(float(la))
        else:                                      # 어느 쪽도 아님 → NaN
            lat_fix.append(float('nan')); lon_fix.append(float('nan'))
    return pd.Series(lat_fix).values, pd.Series(lon_fix).values

def pick_col(df, candidates):
    """후보 리스트 중 df에 실제 존재하는 첫 컬럼명 반환(없으면 None)."""
    return next((c for c in candidates if c in df.columns), None)

# ----------------------------------------------------
# 3) folium 지도 생성(미니멀 + 커스텀 아이콘)
#    - 후보지: ☀️ 아이콘(일사량 컬러맵 반영)
#    - 기존 설비: 패널 아이콘 + 클러스터
#    - 용량 가중 Heatmap 추가
# ----------------------------------------------------
# 기본 지도: 카르토DB 라이트 타일 + 스케일바
m = folium.Map(location=[36.4,128.0], zoom_start=7, tiles="CartoDB positron", control_scale=True)
Fullscreen(position="topleft").add_to(m)

# 후보지 일사량 컬러맵(예시값 기반)
irr_values = [s["일사량"] for s in sites if s.get("일사량") is not None]
vmin, vmax = (min(irr_values), max(irr_values)) if irr_values else (4500, 5100)
solar_cmap = cm.LinearColormap(["#fff7bc","#fec44f","#fe9929","#d95f0e"], vmin=vmin, vmax=vmax)
solar_cmap.caption = "연평균 일사량 (예시)"

# 후보지 레이어
fg_candidates = folium.FeatureGroup(name="후보지", show=True).add_to(m)
CAND_ICON_PX = 56   # 후보지 아이콘 div 크기
CAND_ICON_FZ = 44   # 폰트 아이콘 크기

# 후보지 마커(툴팁만, 팝업은 제거)
for s in sites:
    irr = s.get("일사량")
    color = solar_cmap(irr) if irr is not None else region_color[s["광역"]]

    icon_html = f"""
    <div style="width:{CAND_ICON_PX}px;height:{CAND_ICON_PX}px;display:flex;align-items:center;justify-content:center;pointer-events:auto;">
        <i class="fa-solid fa-sun"
            style="font-size:{CAND_ICON_FZ}px;line-height:1;color:{color};
                    filter:drop-shadow(0 0 1px rgba(0,0,0,.35));"></i>
    </div>
    """
    folium.Marker(
        [s["lat"], s["lon"]],
        tooltip=folium.Tooltip(f"{s['광역']} {s['지역']}", sticky=True),
        icon=DivIcon(
            html=icon_html,
            icon_size=(CAND_ICON_PX, CAND_ICON_PX),
            icon_anchor=(CAND_ICON_PX//2, CAND_ICON_PX//2)
        ),
    ).add_to(fg_candidates)

# 컬러바 추가
solar_cmap.add_to(m)

# 기존 설비 마커 스타일
FAC_ICON_PX = 40
FAC_ICON_FZ = 34

def add_facilities(df, region_name):
    """
    설비 목록 df를 받아:
    - 이름/용량/주소 컬럼을 유연하게 탐색
    - 좌표 보정 후 패널 아이콘 마커 + 클러스터로 지도에 추가
    """
    name_col = pick_col(df, ["발전소명","발전소 명","발전소명칭","사업장명","사업명","시설명","명칭","발전사업명"])
    cap_col  = pick_col(df, ["설비용량(KW)","허가용량(키로와트)","설비용량(MW)","설비용량"])
    addr_col = pick_col(df, ["위치","설치장소","주소","소재지","도로명주소","지번주소"])

    lat, lon = normalize_latlon(df, "위도", "경도")
    grp = folium.FeatureGroup(name=f"기존 설비 ({region_name})", show=True).add_to(m)
    cluster = MarkerCluster().add_to(grp)

    for i, row in df.reset_index(drop=True).iterrows():
        # 좌표 누락은 스킵
        if pd.isna(lat[i]) or pd.isna(lon[i]):
            continue

        # 마커 제목(이름>주소>대체)
        if name_col and pd.notna(row.get(name_col)):
            nm = str(row[name_col]).strip()
        elif addr_col and pd.notna(row.get(addr_col)):
            nm = str(row[addr_col]).strip()
        else:
            nm = f"{region_name} 설비 #{i+1}"

        # 용량/주소 텍스트(현재는 툴팁만 쓰므로 보류 가능)
        cap_val = pd.to_numeric(row.get(cap_col), errors="coerce") if cap_col else None
        cap_txt = f"용량: {cap_val:,.2f} kW<br>" if pd.notna(cap_val) else ""
        addr_txt = f"{row.get(addr_col)}<br>" if addr_col and pd.notna(row.get(addr_col)) else ""

        icon_html = f"""
        <div style="width:{FAC_ICON_PX}px;height:{FAC_ICON_PX}px;display:flex;align-items:center;justify-content:center;pointer-events:auto;">
            <i class="fa-solid fa-solar-panel"
                style="font-size:{FAC_ICON_FZ}px;line-height:1;color:#f59e0b;filter:drop-shadow(0 0 1px rgba(0,0,0,.35));"></i>
        </div>
        """
        folium.Marker(
            [lat[i], lon[i]],
            tooltip=folium.Tooltip(nm, sticky=True),
            icon=DivIcon(
                html=icon_html,
                icon_size=(FAC_ICON_PX, FAC_ICON_PX),
                icon_anchor=(FAC_ICON_PX//2, FAC_ICON_PX//2),
                class_name="fa-divicon"
            ),
        ).add_to(cluster)

# 설비 레이어 추가
add_facilities(df_gb, "경북")
add_facilities(df_gn, "경남")
add_facilities(df_cn, "충남")

# 설비 밀집도 Heatmap(용량 가중치: 로그 정규화 → 0~1 스케일링 + 최소 0.1)
all_df = pd.concat([df_gb, df_gn, df_cn], ignore_index=True)
lat_h, lon_h = normalize_latlon(all_df, "위도", "경도")
cap_col_all = pick_col(all_df, ["설비용량(KW)","허가용량(키로와트)"])
if cap_col_all:
    weights_raw = pd.to_numeric(all_df[cap_col_all], errors="coerce")
    weights = np.log1p(weights_raw.fillna(weights_raw.median()))
    if weights.max() > 0:
        weights = (weights - weights.min()) / (weights.max() - weights.min()) + 0.1
else:
    weights = pd.Series([1]*len(all_df))

heat = [
    [lat_h[i], lon_h[i], float(weights.iloc[i])]
    for i in range(len(lat_h))
    if pd.notna(lat_h[i]) and pd.notna(lon_h[i])
]
HeatMap(heat, name="밀집도(용량가중)", radius=18, blur=22, min_opacity=0.25, show=False).add_to(m)

# 레이어 컨트롤(우상단)
folium.LayerControl(collapsed=True, position="topright").add_to(m)

# ----------------------------------------------------
# 4) 데이터 모음: 2024 버블 차트 + 라인 차트들
#    - 버블: 일사량(MJ/m²) vs 설비이용률(%) (버블 크기=설비용량)
#    - 라인: 발전량/설비용량/일사량 (경북/경남/충남)
#    - folium 지도 HTML 문자열(map_html)과 함께, 모두 하나의 HTML 템플릿에 삽입
# ----------------------------------------------------
map_html = m.get_root().render()  # folium → HTML 문자열

# (1) 버블 차트용 데이터
df_all = pd.read_excel("dataset/최종일사량합친데이터.xlsx")
df_2024 = df_all[df_all["연도"] == 2024].copy()

# 숫자 열 견고화(문자/공백/'-' 처리 → 숫자)
for col in ["설비용량(MW)", "일사량(MJ/m2)", "설비이용률(%)"]:
    df_2024[col] = (
        df_2024[col].astype(str).str.replace(",", "", regex=False).str.strip()
        .replace({"": np.nan, "-": np.nan})
    )
    df_2024[col] = pd.to_numeric(df_2024[col], errors="coerce")

# 버블 크기: 최소 50 보장
df_2024["설비용량"] = df_2024["설비용량(MW)"].fillna(0).apply(lambda v: max(50, float(v)))

# 축 범위/평균선 계산
x = pd.to_numeric(df_2024["일사량(MJ/m2)"], errors="coerce")
y = pd.to_numeric(df_2024["설비이용률(%)"], errors="coerce")
xmin, xmax = float(x.min()), float(x.max())
ymin, ymax = float(y.min()), float(y.max())
padx, pady = (xmax - xmin) * 0.05, (ymax - ymin) * 0.05
mean_x, mean_y = x.mean(), y.mean()

# 지역 색상 맵(디자인 고정)
color_map = {
    "서울": "#E74C3C","부산": "#3498DB","대구": "#2ECC71","인천": "#F39C12","광주": "#9B59B6",
    "대전": "#1ABC9C","울산": "#E67E22","경기": "#34495E","강원": "#F1C40F","경북": "#E91E63",
    "경남": "#00BCD4","전북": "#FF9800","전남": "#795548","충북": "#607D8B","충남": "#4CAF50","제주": "#673AB7"
}

# 버블 차트 생성
fig = px.scatter(
    df_2024,
    x="일사량(MJ/m2)", y="설비이용률(%)",
    size="설비용량", color="지역", hover_name="지역",
    size_max=60, title="2024년 지역별 일사량과 설비이용률 비교 분석",
    color_discrete_map=color_map
)
fig.update_traces(text=None)

# 라벨 전략: 큰 버블(설비용량≥THRESH)은 고정 라벨, 작은 버블은 화살표 라벨
THRESH = 500
for _, row in df_2024.iterrows():
    if row["설비용량"] >= THRESH:
        fig.add_annotation(
            x=row["일사량(MJ/m2)"], y=row["설비이용률(%)"],
            text=f"<b>{row['지역']}</b>", showarrow=False,
            font=dict(color="white", size=11)
        )
    else:
        fig.add_annotation(
            x=row["일사량(MJ/m2)"], y=row["설비이용률(%)"],
            text=row["지역"], showarrow=True, arrowhead=2,
            ax=15, ay=-15, font=dict(color="black", size=11),
            bgcolor="rgba(255,255,255,0.6)"
        )

# 평균선(사분면) 추가 + 레이아웃
fig.update_layout(autosize=True, height=720, template="plotly_white")
fig.add_shape(type="line", x0=mean_x, y0=ymin, x1=mean_x, y1=ymax, line=dict(color="red", dash="dash", width=2))
fig.add_shape(type="line", x0=xmin, y0=mean_y, x1=xmax, y1=mean_y, line=dict(color="red", dash="dash", width=2))
fig.add_annotation(x=xmax, y=ymax, text="성과 우수 (자원↑, 효율↑)", showarrow=False,
                   font=dict(color="green"), xanchor="right", yanchor="bottom")
fig.update_xaxes(range=[xmin - padx, xmax + padx], title="일사량 (MJ/m²)")
fig.update_yaxes(range=[ymin - pady, ymax + pady + 0.2], title="설비이용률 (%)")

# 버블 차트 HTML 조각(동일 페이지에 삽입)
chart_html = fig.to_html(full_html=False, include_plotlyjs="cdn", config={"displaylogo": False})

# (2) 라인 차트(발전량/설비용량/일사량) - 경북/경남/충남 비교
# merged가 외부에서 이미 만들어져 있으면 그대로 사용, 아니면 df_all로 대체
try:
    merged
except NameError:
    merged = df_all.copy()

target_regions = ["경북", "경남", "충남"]
color_map3 = {"경남": "#1f77b4", "경북": "#d62728", "충남": "#2ca02c"}

# ---- (a) 발전량 추이
df_filtered_gc = merged[merged["지역"].isin(target_regions)].copy()
df_filtered_gc["연도"] = pd.to_numeric(df_filtered_gc["연도"], errors="coerce")
df_grouped_gc = (
    df_filtered_gc.groupby(["연도", "지역"])[["발전량(MWh)", "설비용량(MW)"]]
    .sum().reset_index()
)

fig_gen = go.Figure()
for region in target_regions:
    sub = df_grouped_gc[df_grouped_gc["지역"] == region]
    fig_gen.add_trace(go.Scatter(
        x=sub["연도"], y=sub["발전량(MWh)"],
        mode="lines+markers", name=region,
        line=dict(color=color_map3[region], width=3),
        marker=dict(size=9, symbol="circle")
    ))
fig_gen.update_layout(
    title="경북·경남·충남 연도별 발전량 추이",
    template="plotly_white",
    xaxis=dict(title="연도", dtick=1, showgrid=True, gridcolor="lightgray"),
    yaxis=dict(title="발전량(MWh)", showgrid=True, gridcolor="lightgray"),
    legend=dict(title="지역", bgcolor="rgba(255,255,255,0.7)", bordercolor="lightgray", borderwidth=1),
    height=420
)

fig_gen.update_layout(width=1200,height=550)

chart_html_gen = fig_gen.to_html(
    full_html=False, include_plotlyjs=False,
    config={"displaylogo": False, "responsive": True},
    default_width="100%", default_height=460
)

# ---- (b) 설비용량 추이
fig_cap = go.Figure()
for region in target_regions:
    sub = df_grouped_gc[df_grouped_gc["지역"] == region]
    fig_cap.add_trace(go.Scatter(
        x=sub["연도"], y=sub["설비용량(MW)"],
        mode="lines+markers", name=region,
        line=dict(color=color_map3[region], width=3, dash="dot"),
        marker=dict(size=8, symbol="square")
    ))
fig_cap.update_layout(
    title="경북·경남·충남 연도별 설비용량 추이",
    template="plotly_white",
    xaxis=dict(title="연도", dtick=1, showgrid=True, gridcolor="lightgray"),
    yaxis=dict(title="설비용량(MW)", showgrid=True, gridcolor="lightgray"),
    legend=dict(title="지역", bgcolor="rgba(255,255,255,0.7)", bordercolor="lightgray", borderwidth=1),
    height=420
)

fig_cap.update_layout(width=1200,height=550)

chart_html_cap = fig_cap.to_html(
    full_html=False, include_plotlyjs=False,
    config={"displaylogo": False, "responsive": True},
    default_width="100%", default_height=460
)

# ---- (c) 일사량 추이(평균)
df_filtered_ir = merged[merged["지역"].isin(target_regions)].copy()
df_filtered_ir["연도"] = pd.to_numeric(df_filtered_ir["연도"], errors="coerce")
df_filtered_ir = df_filtered_ir.sort_values("연도")
df_grouped_ir = (
    df_filtered_ir.groupby(["연도", "지역"])["일사량(MJ/m2)"]
    .mean().reset_index()
)
fig_irr = px.line(
    df_grouped_ir, x="연도", y="일사량(MJ/m2)", color="지역",
    markers=True, title="경북·경남·충남 연도별 일사량 추이",
    color_discrete_map={"경북": "#d62728", "경남": "#1f77b4", "충남": "#2ca02c"},
)
fig_irr.update_traces(line=dict(width=3), marker=dict(size=9))
fig_irr.update_layout(
    template="plotly_white",
    xaxis=dict(title="연도", dtick=1, showgrid=True, gridcolor="lightgray"),
    yaxis=dict(title="일사량 (MJ/m²)", showgrid=True, gridcolor="lightgray"),
    legend=dict(title="지역", bgcolor="rgba(255,255,255,0.7)", bordercolor="lightgray", borderwidth=1),
    height=420
)

fig_irr.update_layout(width=1200,height=550)

chart_html_irr = fig_irr.to_html(
    full_html=False, include_plotlyjs=False,
    config={"displaylogo": False, "responsive": True},
    default_width="100%", default_height=460
)

# ----------------------------------------------------
# 5) 단일 HTML 템플릿(사이드바/본문/그리드/맵 래퍼)
#    - folium 지도와 plotly 차트를 iframe 없이 한 페이지에 통합
#    - 반응형 레이아웃 + 후보지 요약(장단점) 섹션 포함
# ----------------------------------------------------
template = """<!doctype html>
<html lang="ko">
<head>
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.2/css/all.min.css">
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>Enerlystic | Solar Atlas</title>
<link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;600;800&display=swap" rel="stylesheet">
<style>
:root{{
  --brand:#16A34A;   /* green-600 */
  --brand2:#166534;  /* green-800 */
  --bg1:#F0FDF4;     /* green-50 */
  --bg2:#DCFCE7;     /* green-100 */
  --accent:#0EA5E9;  /* sky-500 */
}}

*{{box-sizing:border-box}}
body{{
    margin:0;
    font-family:'Poppins',system-ui,-apple-system,sans-serif;
    color:#0f172a;
    background:
        radial-gradient(60% 45% at 15% 0%, rgba(255,186,90,.18) 0%, rgba(255,186,90,0) 60%),
        radial-gradient(50% 40% at 90% 10%, rgba(14,165,233,.12) 0%, rgba(14,165,233,0) 60%),
        linear-gradient(180deg, var(--bg1), var(--bg2));
}}
a{{color:inherit;text-decoration:none}}
.layout{{display:grid;grid-template-columns:320px 1fr;min-height:100vh}}

.sidebar{{
    background:linear-gradient(180deg,var(--brand),var(--brand2));
    color:#fff;
    padding:28px 22px;
    position:sticky; top:0; height:100vh;
    box-shadow: 0 10px 30px rgba(0,0,0,.12);
}}
.logo{{font-weight:800;font-size:42px;letter-spacing:.5px;margin-bottom:28px}}

.sidebar .nav-item{{
    display:block; margin:26px 0 18px; padding:10px 12px;
    font-weight:700; font-size:20px; line-height:1.2;
    color:#fff; border-radius:12px;
    text-shadow:0 1px 1px rgba(0,0,0,.18);
    transition: background .2s ease, transform .2s ease;
}}
.sidebar .nav-item small{{
    display:block; margin-top:6px;
    font-weight:600; font-size:13px; line-height:1.3;
    color:rgba(255,255,255,.95);
}}
.sidebar .nav-item:hover{{ background:rgba(255,255,255,.14); transform: translateY(-1px); }}

.content{{padding:28px 36px}}
.header h1{{margin:0;font-size:32px;font-weight:800;letter-spacing:.5px}}
.header p{{margin:8px 0 22px;color:var(--muted)}}

.card{{
    background:var(--card);
    border-radius:18px;
    border:1px solid rgba(0,0,0,.05);
    box-shadow:0 12px 28px rgba(0,0,0,.08);
    padding:20px
}}

.map-card{{padding:0;overflow:hidden}}
.map-card .map-wrapper{{width:100%;height:calc(100vh - 180px)}}
/* 지도 높이 전파 (folium 상위/하위 div 모두 100%) */
.map-card .map-wrapper > div{{height:100%}}
.map-card .map-wrapper .folium-map{{height:100% !important}}
.map-card .map-wrapper > div > div{{height:100%}}
.map-card .map-wrapper > div > div > div{{height:100%}}

.badge{{
    display:inline-block;background:#FFE4C7;color:#7C2D12;
    padding:6px 10px;border-radius:999px;font-size:12px;font-weight:700
}}
.anchor{{scroll-margin-top:18px}}

@media(max-width:1100px){{
    .map-card .map-wrapper{{height:70vh}}
    .layout{{grid-template-columns:1fr}}
    .sidebar{{position:relative;height:auto;border-radius:0 0 18px 18px}}
}}

/* === 라인차트 그리드 (2열 → 반응형) === */
.charts-grid{{
    margin-top:16px;
    padding:14px;
    display:grid;
    grid-template-columns: repeat(2, minmax(420px, 1fr));
    gap:16px;
}}
.charts-grid .chart{{ min-height:460px; }}
.charts-grid .chart > div{{ width:100% !important; height:100% !important; }}

/* 특정 차트를 가로 전체로 펼치고 싶을 때 */
.charts-grid .span-2{{ grid-column: 1 / -1; }}

@media (max-width: 1400px){{
  .charts-grid{{ grid-template-columns: 1fr; }} /* 화면이 좁아지면 1열 */
}}

/* 후보지 요약 줄(지역명 있는 라인)만 크게 */
.site-summary {{ 
    font-size: 18px;
    line-height: 1.6;
}}
.site-summary .site-name {{ 
    font-size: 20px;
    font-weight: 800;
}}
</style>
</head>
<body>
<div class="layout">
    <aside class="sidebar">
        <div class="logo">☀️ Enerlystic</div>
        <a class="nav-item" href="#data">데이터 모음<small>2024 데이터 자료 비교 분석</small></a>
        <a class="nav-item" href="#plants">태양광 발전소 현황<small>발전소 현황 및 후보지 시각화</small></a>
        <a class="nav-item" href="#candidates">태양광 후보지 발전소<small>장•단점 요약</small></a>
    </aside>

    <main class="content">

    <!-- 데이터 모음 -->
    <div id="data" class="anchor"></div>
    <div class="header">
        <span class="badge">데이터 모음</span>
        <h1>2024년 지역별 비교</h1>
        <p>일사량(MJ/m²) vs 설비이용률(%) 버블 차트</p>
    </div>
    <section class="card" style="padding:10px 10px 0">
        {chart_html}
    </section>

    <!-- 라인차트 그리드 (발전량/설비용량/일사량) -->
    <section class="card charts-grid">
        <div class="chart">{chart_html_gen}</div>
        <div class="chart">{chart_html_cap}</div>
        <div class="chart span-2">{chart_html_irr}</div>
    </section>

    <!-- 태양광 발전소 현황: 지도 -->
    <div id="plants" class="anchor"></div>
    <div class="header" style="margin-top:24px">
        <span class="badge">태양광 발전소 현황</span>
        <h1>경북, 경남, 충남 설비 현황 및 후보지 추천</h1>
        <p>기존 설비 현황 및 추천 지역 확인</p>
    </div>
    <section class="card map-card">
        <div class="map-wrapper">{map_html}</div>
    </section>

    <!-- 태양광 후보지 발전소: 후보지 요약 -->
    <div id="candidates" class="anchor"></div>
    <div class="header" style="margin-top:24px">
        <span class="badge">태양광 후보지 발전소</span>
        <h1>후보지 요약</h1>
        <p> <p>
    </div>
    <section class="card">
        <ul style="margin:6px 0 0 18px;line-height:1.8">
        {site_list_items}
        </ul>
    </section>

    </main>
</div>
</body>
</html>
"""

# ----------------------------------------------------
# 6) 후보지 장단점(요약 블록)
#    - site_details 사전 → HTML 리스트로 변환하여 템플릿에 삽입
# ----------------------------------------------------
site_details = {
    "경북 포항시 남구": {
        "pros": [
            "산업단지 활용 가능 → 전력 수요와 직접 연결 → 송배전 효율 ↑.",
            "동해안에 위치해 신항만·산업 기반과 연계 가능성 큼."
        ],
        "cons": [
            "해안 지역 → 태풍, 염해(바닷바람에 의한 부식) 리스크 존재.",
            "산업지 토지 비용이 높을 수 있고, 용도 변경 절차 복잡."
        ],
    },
    "경남 김해시": {
        "pros": [
            "부산권 대도시와 매우 가까워 전력 소비지 인접성 최고.",
            "도시와 농업지역이 혼재 → 다양한 형태의 태양광 에너지 활용 가능"
        ],
        "cons": [
            "도시 확장으로 인한 토지 용도 제한 가능성."
        ],
    },
    "경북 구미시": {
        "pros": [
            "낙동강 평지 → 대규모 설비 부지 확보 용이.",
            "구미 국가산업단지 전력 수요지 인접 → 산업용 전력 직접 공급 가능.",
            "내륙 입지로 기후 안정성 높음."
        ],
        "cons": [
            "산업단지 주변은 환경 규제(경관·수질 등) 고려 필요."
        ],
    },
    "경남 창녕군": {
        "pros": [
            "농업·축산 중심 지역 → 주민참여형 태양광(영농형, 축사형 등) 확장성 큼."
        ],
        "cons": [
            "농업 중심지라 주민 수용성 확보 필요."
        ],
    },
    "충남 논산시": {
        "pros": [
            "넓은 농업 평지 → 대규모 태양광 단지 개발 유리.",
            "군부대·농업지 중심 지역이라 토지 확보 상대적으로 용이."
        ],
        "cons": [
            "농업지 활용 시 주민 민원(경관·농지 전용)이 발생할 수 있음."
        ],
    },
    "충남 예산군": {
        "pros": [
            "삽교천 유역 평야로 평탄지 많음 → 설치 용이.",
            "수도권과 비교적 가까워 송전 부담 적음.",
            "충남도가 신재생에너지 정책을 적극 추진하는 지역."
        ],
        "cons": [
            "삽교천 유역은 홍수·침수 리스크 존재"
        ],
    },
}

def _fmt_irr(v):
    """일사량 숫자를 '1,234 MJ/m²' 형태로 포맷"""
    try:
        return f"{int(v):,} MJ/m²"
    except Exception:
        return "-"

def _fmt_latlon(site, decimals=5):
    """좌표 표시(위도, 경도) 안전 포맷"""
    try:
        return f"{float(site['lat']):.{decimals}f}, {float(site['lon']):.{decimals}f}"
    except Exception:
        return "-"

def build_site_list_items(sites, details_dict):
    """
    후보지 배열(sites)과 장단점 사전(details_dict)을 HTML <li> 리스트로 변환.
    - summary 라인에 지역명·좌표·일사량 표시
    - details 블록에 장점/단점 UL 삽입
    """
    items = []
    for s in sites:
        key    = f"{s['광역']} {s['지역']}"
        coords = _fmt_latlon(s)
        irr_txt= _fmt_irr(s.get("일사량"))

        d = details_dict.get(key, {})
        pros = d.get("pros", [])
        cons = d.get("cons", [])

        pros_html = "".join(f"<li>{p}</li>" for p in pros)
        cons_html = "".join(f"<li>{c}</li>" for c in cons)

        items.append(f"""
<li style="margin:10px 0">
    <details open>
        <summary class="site-summary">
            <span class="site-name">{key}</span>
            — 좌표: {coords} (일사량: {irr_txt})
        </summary>
        {"<div style='margin-top:8px'><b>✅ 장점</b><ul style='margin:6px 0 8px 20px'>" + pros_html + "</ul></div>" if pros else ""}
        {"<div style='margin-top:4px'><b>⚠️ 단점</b><ul style='margin:6px 0 0 20px'>" + cons_html + "</ul></div>" if cons else ""}
    </details>
</li>""")
    return "\n".join(items)

# 후보지 요약 HTML 생성
site_list_items = build_site_list_items(sites, site_details)

# ----------------------------------------------------
# 7) 템플릿에 동적 조각 삽입 → 단일 HTML 파일 생성 & 열기
# ----------------------------------------------------
html = template.format(
    site_list_items=site_list_items,
    map_html=map_html,
    chart_html=chart_html,            # 버블 차트
    chart_html_gen=chart_html_gen,    # 발전량
    chart_html_cap=chart_html_cap,    # 설비용량
    chart_html_irr=chart_html_irr     # 일사량
)

dashboard_path = "index.html"
with open(dashboard_path, "w", encoding="utf-8") as f:
    f.write(html)

print("생성 완료:", dashboard_path)
webbrowser.open(dashboard_path)


생성 완료: index.html


True